# Python Problem
To write a python script to fetch all the tweets(as many as allowed by Twitter API) done by midas@IIITD twitter handle and dump the responses into JSONlines file. Further, parse these JSONline files to display the following for every tweet in a tabular format.

* The text of the tweet.
* Date and time of the tweet.
* The number of favorites/likes.
* The number of retweets.
* Number of Images present in Tweet. If no image returns None.


---



## Installing the required Libraries

In [1]:
#!pip install jsonlines

##Importing Libraries & Setting up API Credentials

In [0]:
import jsonlines
import tweepy

credentials = {'consumer_key':'',
               'consumer_secret':'',
               'access_token':'',
               'access_token_secret':''}

auth = tweepy.OAuthHandler(credentials['consumer_key'], credentials['consumer_secret'])
auth.set_access_token(credentials['access_token'], credentials['access_token_secret'])
api = tweepy.API(auth)

## Scrapping Tweets from @midasIIITD and Writting them to JSONlines File

In [0]:
tweets = api.user_timeline(screen_name='midasiiitd',count=100)
tweets = [i._json for i in tweets]
with open('output.jsonl', 'w') as fp:
  writer = jsonlines.Writer(fp)
  writer.write_all(tweets)